In [ ]:
import numpy as np
from math import *
import numba
import random

import matplotlib.pyplot as plt

In [ ]:
def LCA(num_alternatives, k, v, beta, a, start_time, max_time, dt=1e-3, sigma = 1.0):

	time = start_time

	y = [0] * num_alternatives
	y_data = []
	for i in range(num_alternatives):
		y_data.append([0])

	while time < max_time:

		for i in range(num_alternatives):
			#get I
			I = beta * (sum(y)-y[i])

			z = np.random.normal(loc=0, scale=sigma)
			y[i] += (v[i] - I - k*y[i])*dt + sigma * (np.sqrt(dt)*z)
			y[i] = max(y[i], 0)

			y_data[i].append(y[i])

			#update time
			time += dt


		#exit loop if decision boundry met
		if max(y)>=a:
			return np.array(y_data), y.index(max(y))
			
	#return if no decision made
	return np.array(y_data), -1

In [ ]:
def FFI(J:int, v:np.ndarray, a:float, alpha:float, x0:np.ndarray, tau:float = 0.0, scale:float = 1.0):
	"""
	J: Number of alternatives
	v: drift rates of all alternatives
	a: threshold parameter
	alpha: strength of feedforward inhibition
	x0: starting points for all alternatives

	tau = 0.0: starting time (non-decision time)
	scale = 1.0: sigma, or scale coefficient
	"""
	time = start_time

	y = [0] * num_alternatives
	y_data = []
	for i in range(num_alternatives):
		y_data.append([0])

	while time < max_time:

		for i in range(num_alternatives):
			#get I
			I = beta * (sum(y)-y[i])

			z = np.random.normal(loc=0, scale=sigma)
			y[i] += (v[i] - I - k*y[i])*dt + sigma * (np.sqrt(dt)*z)
			y[i] = max(y[i], 0)

			y_data[i].append(y[i])

			#update time
			time += dt


		#exit loop if decision boundry met
		if max(y)>=a:
			return np.array(y_data), y.index(max(y))
			
	#return if no decision made
	return np.array(y_data), -1